In [1]:
import numpy as np
import pandas as pd

import pickle as pkl

import gc
from sklearn.model_selection import KFold
from cuml.metrics import mean_absolute_error, mean_squared_error

import seaborn as sns

In [2]:
out_base_path = '/media/hiroki/working/kaggle/trends-neuroimaging/models'

In [3]:
OUTPUT_DICT = ''

ID = 'Id'
TARGET_COLS = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
SEED = 2020

N_FOLD = 5

In [4]:
#base_path2 = '/media/hiroki/share/kaggle_data/trends-assessment-prediction/tf_data'
#fnc = pd.read_csv(base_path2+'/pca'+'/fnc_pca.csv', dtype={'Id':str})
#loading = pd.read_csv(base_path2+'/ica'+'/loading_ica.csv', dtype={'Id':str})

In [5]:
base_path = '/media/hiroki/share/kaggle_data/trends-assessment-prediction/'
train = pd.read_csv(base_path+'train_scores.csv', dtype={'Id':str})\
            #.dropna().reset_index(drop=True) # to make things easy
reveal_ID = pd.read_csv(base_path+'reveal_ID_site2.csv', dtype={'Id':str})
ICN_numbers = pd.read_csv(base_path+'ICN_numbers.csv')
loading = pd.read_csv(base_path+'loading.csv', dtype={'Id':str})
fnc = pd.read_csv(base_path+'fnc.csv', dtype={'Id':str})
sample_submission = pd.read_csv(base_path+'sample_submission.csv', dtype={'Id':str})

In [6]:
ic_loading = pd.read_csv("/media/hiroki/working/kaggle/trends-neuroimaging/tf_data/ica/loading_ica.csv", dtype={'Id':str})
pc_fnc = pd.read_csv("/media/hiroki/working/kaggle/trends-neuroimaging/tf_data/pca/fnc_pca.csv", dtype={'Id':str})

In [7]:
fnc_features, loading_features = list(fnc.columns[1:]), list(loading.columns[1:])

In [8]:
sample_submission['ID_num'] = sample_submission[ID].apply(lambda x: int(x.split('_')[0]))
test = pd.DataFrame({ID: sample_submission['ID_num'].unique().astype(str)})
del sample_submission['ID_num']; gc.collect()
test.head()

,Id
0,10003
1,10006
2,10010
3,10011
4,10012


In [9]:
test.head()

,Id
0,10003
1,10006
2,10010
3,10011
4,10012


In [10]:
# merge
train = train.merge(loading, on=ID, how='left')
train = train.merge(fnc, on=ID, how='left')
train = train.merge(ic_loading, on=ID, how='left')
train = train.merge(pc_fnc, on=ID, how='left')
train

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,IC_01,IC_07,IC_05,IC_16,...,fnc_pc140,fnc_pc141,fnc_pc142,fnc_pc143,fnc_pc144,fnc_pc145,fnc_pc146,fnc_pc147,fnc_pc148,fnc_pc149
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998,0.006070,0.014466,0.004136,0.000658,...,-0.152955,0.219027,0.001179,-0.058451,-0.388662,-0.241710,-0.224239,0.056872,0.079370,-0.029729
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361,0.009087,0.009291,0.007049,-0.002076,...,0.005318,-0.118583,-0.179065,-0.381546,-0.060484,-0.037991,-0.434464,0.299034,0.079064,-0.250063
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641,0.004675,0.000957,0.006154,-0.000429,...,0.041355,-0.013931,-0.005913,0.202989,0.099073,0.028089,-0.011903,0.235919,-0.014796,-0.277931
3,10005,66.532630,NaN,NaN,52.108977,69.993075,-0.000398,0.006878,0.009051,0.000369,...,0.003979,-0.372342,0.207945,-0.128819,0.251851,-0.114924,0.014899,0.494779,-0.054188,-0.176513
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421,0.005192,0.010585,0.012160,-0.000920,...,0.248737,-0.070015,0.154606,-0.153220,0.120113,-0.199256,-0.132451,0.268803,-0.344660,-0.295151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21746,14.257265,21.358872,61.165998,51.778483,54.640179,-0.001115,0.007108,0.008652,0.003596,...,0.040849,0.125464,-0.034158,0.092616,-0.071046,0.194730,0.154677,-0.083255,-0.088222,-0.018031
5873,21747,55.456978,68.169675,29.907995,55.349257,54.019517,0.007263,0.016489,0.012704,0.004357,...,-0.179107,-0.066442,0.119179,-0.021575,0.195725,-0.186029,0.108386,-0.323034,0.055834,0.079152
5874,21750,48.948756,55.114811,60.878271,38.617246,50.679885,0.005996,0.003873,0.012353,0.000242,...,-0.288717,-0.264649,-0.196736,-0.136057,0.357413,0.516544,0.013619,0.252955,0.160745,0.030227
5875,21752,66.532630,59.844808,72.303110,55.458281,46.870235,0.000627,0.011407,0.010957,0.000534,...,0.381169,0.085059,-0.440243,0.101484,0.088927,-0.123115,0.510575,-0.071335,-0.245805,-0.324772


In [11]:
# merge
test = test.merge(loading, on=ID, how='left')
test = test.merge(fnc, on=ID, how='left')
test = test.merge(ic_loading, on=ID, how='left')
test = test.merge(pc_fnc, on=ID, how='left')
test

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,...,fnc_pc140,fnc_pc141,fnc_pc142,fnc_pc143,fnc_pc144,fnc_pc145,fnc_pc146,fnc_pc147,fnc_pc148,fnc_pc149
0,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,...,-0.270566,-0.071504,0.190468,0.097928,-0.165759,0.202768,0.185363,-0.172570,0.180270,0.120484
1,10006,0.000334,0.005311,0.010053,0.006920,-0.000065,0.015310,0.016543,0.004794,0.003982,...,-0.044184,0.051582,-0.014007,0.184597,0.160777,-0.119761,-0.015031,0.085200,-0.089972,-0.169212
2,10010,0.007103,0.006144,0.009770,-0.002884,-0.001346,0.015651,0.011613,-0.003291,0.013423,...,-0.087201,-0.044116,-0.153219,-0.254180,-0.277504,0.185504,-0.079110,0.084088,0.078964,0.073566
3,10011,0.004362,0.010240,0.010167,0.004492,-0.001623,0.017381,0.014680,0.007453,0.008786,...,0.025658,0.205716,-0.131292,0.091335,0.414347,-0.060277,0.079852,-0.162485,-0.099584,0.134600
4,10012,-0.007521,-0.003918,0.008434,-0.001145,0.002017,0.015065,0.019616,0.004140,-0.003744,...,0.668838,-0.542525,-0.331014,0.130991,-0.024522,-0.317343,-0.363590,0.340918,0.335200,0.044452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21745,0.005406,0.006275,0.012252,0.003518,0.001400,0.015054,0.015373,0.001532,0.003546,...,-0.027194,0.189567,0.050031,0.145211,0.599115,0.308601,-0.151217,0.125993,-0.410924,-0.190956
5873,21748,0.004240,0.009213,0.010981,0.000443,-0.003072,0.010702,0.014673,0.005523,0.005780,...,0.125925,0.119710,0.427992,0.346669,0.102869,0.050158,0.172364,0.557885,0.170517,-0.061226
5874,21749,0.004783,0.017910,0.012128,-0.005683,-0.011613,0.017000,0.007230,0.001315,0.008788,...,-0.219974,-0.289622,0.013956,-0.522414,-0.192238,0.028535,0.040753,0.239192,-0.050967,0.116374
5875,21751,0.003835,0.015067,0.015428,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,...,-0.200811,-0.126997,0.073071,-0.389314,0.116944,-0.080405,-0.344519,0.376230,-0.503913,0.371715


In [12]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [13]:
def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [14]:
# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
#FNC_SCALE = 1/600

#train[fnc_features] *= FNC_SCALE
#test[fnc_features] *= FNC_SCALE

In [15]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(train_x, train_y, test_x, model, target_name):
    preds = []
    preds_test = []
    va_idxes = []
    
    score = []
    mae = []
    rmse = []

    # shuffleしなくても良い
    kf = KFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx].values, train_x.iloc[va_idx].values
        tr_y, va_y = train_y.iloc[tr_idx].values, train_y.iloc[va_idx].values
        model.fit(tr_x, tr_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
        
        score.append(metric(va_y, pred))
        #mae.append(mean_absolute_error(va_y, pred))
        #rmse.append(np.sqrt(mean_squared_error(va_y, pred)))
        
    score_cv = np.array(score).mean()
    #mae_cv = np.array(mae).mean()
    #rmse_cv = np.array(rmse).mean()
    print("{0}_score:{1}".format(target_name, np.round(score_cv, 8)))
    #print("{0}_mae:{1}".format(target_name, np.array(mae_cv).mean()))
    #print("{0}_rmse:{1}".format(target_name, np.array(rmse_cv).mean()))
    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test, score_cv

# 1st layer

## RF

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
pred_train_targets_knr = {}
pred_test_targets_knr = {}

pred_train_targets_dfs = {}
pred_test_targets_dfs = {}

overal_score = 0

print("N_FOLD:{}".format(N_FOLD))

for target, depth in [("age", 30),
                      ("domain1_var1", 30),
                      ("domain1_var2", 30),
                      ("domain2_var1", 30),
                      ("domain2_var2", 30)]:
    
    train_df = train[train[target].notnull()]
    test_df = test
        
    use_idx = train_df.index
    train_x = train_df.drop([ID]+TARGET_COLS, axis=1).astype("float32")
    train_y = train_df[target].astype("float32")
    test_x = test_df.drop(ID, axis=1).astype("float32")
    rf = RandomForestRegressor(max_depth=depth)
    
    print("-----{}-----".format(target))
    pred_train, preds_test, score_cv = predict_cv(train_x, train_y, test_x, rf, target)
    overal_score += w*score_cv
    pred_train_targets[target] = pred_train
    pred_test_targets[target] = preds_test
    
    pred_train_targets_dfs[target] = pd.Series(pred_train, name="pre_rf_{}".format(target), index=use_idx)
    pred_train_targets_dfs[target] = pd.merge(train['Id'],
                                              pred_train_targets_dfs[target],
                                              left_index=True,
                                              right_index=True)
    pred_test_targets_dfs[target] = pd.Series(preds_test, name="pre_rf_{}".format(target))
    pred_test_targets_dfs[target] = pd.concat([test['Id'], pred_test_targets_dfs[target]], axis=1)
    
    
print('--------------------------------------------')
print("Overal score:", np.round(overal_score, 8))
#200:0.17859491

N_FOLD:5
-----age-----
age_score:0.16426939


NameError: name 'w' is not defined

In [22]:
pred_test_targets

{}

In [ ]:
for df in pred_train_targets_knr_dfs.values():
    display(df.describe())

In [ ]:
for df in pred_train_targets_knr_dfs.values():
    display(df)

In [ ]:
for i, pred_df in enumerate(pred_train_targets_knr_dfs.values()):
    if i == 0:
        pred_train_dfs = pred_df
    else:
        pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

In [23]:
for i, pred_df in enumerate(pred_test_targets_knr_dfs.values()):
    if i == 0:
        pred_test_dfs = pred_df
        #display(pred_train_dfs, head=False)
    else:
        pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)
        #display(pred_train_dfs, head=False)

In [24]:
pred_train_dfs

,Id,pre_knr_age,pre_knr_domain1_var1,pre_knr_domain1_var2,pre_knr_domain2_var1,pre_knr_domain2_var2
0,10001,58.579739,52.227528,59.565750,47.277256,50.603771
1,10002,57.168205,52.395531,59.227043,47.784794,50.775257
2,10004,51.257458,52.518513,58.674698,47.557449,51.710136
3,10005,57.543938,NaN,NaN,49.855473,51.064487
4,10007,42.947926,49.985779,60.776917,45.188019,52.591808
...,...,...,...,...,...,...
5872,21746,44.743279,52.174969,59.816620,46.257805,50.213261
5873,21747,52.211586,51.370998,61.022194,48.501656,51.463165
5874,21750,48.756733,51.304272,60.420307,46.952251,51.790058
5875,21752,52.345413,52.895702,58.890129,49.035107,51.604050


In [25]:
#pred_train_dfs.to_csv('/media/hiroki/working/kaggle/trends-neuroimaging/models/1st_layer/knr/knr_train.csv', index=False)
#pred_test_dfs.to_csv('/media/hiroki/working/kaggle/trends-neuroimaging/models/1st_layer/knr/knr_test.csv', index=False)